In [5]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
import os
import wandb #library for tracking and visualization


wandb.login()
os.environ['WANDB_NOTEBOOK_NAME'] = '/dos/MIT-WHOI/github_repos/syn_model/scripts/syn_model_gyptorch.ipynb'

Error in callback <function _WandbInit._resume_backend at 0x7f91d2f39790> (for pre_run_cell):


Exception: The wandb backend process has shutdown

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Error in callback <function _WandbInit._pause_backend at 0x7f91d2f39670> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [4]:


wandb.init(
    project = "syn_model",
    notes = "using gyptorch",
    tags = ["gpytorch","gpr","syn"]) #Initializes a new W&B run. Each run is single execution of the training script.

file_path = "/D/MIT-WHOI/github_repos/syn_model/data/dfmerged_dailysynenv.csv"

wandb.log_artifact(file_path, name = "syn_data",type = "dataset")

Error in callback <function _WandbInit._resume_backend at 0x7f91d2f39790> (for pre_run_cell):


Exception: The wandb backend process has shutdown

Problem at: /tmp/ipykernel_21928/3687882060.py 1 <cell line: 1>


Traceback (most recent call last):
  File "/home/mira/anaconda3/envs/gpytorch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 999, in init
    run = wi.init()
  File "/home/mira/anaconda3/envs/gpytorch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 494, in init
    self._wl._global_run_stack[-1].finish()
  File "/home/mira/anaconda3/envs/gpytorch/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 256, in wrapper
    return func(self, *args, **kwargs)
  File "/home/mira/anaconda3/envs/gpytorch/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 222, in wrapper
    return func(self, *args, **kwargs)
  File "/home/mira/anaconda3/envs/gpytorch/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 1678, in finish
    return self._finish(exit_code, quiet)
  File "/home/mira/anaconda3/envs/gpytorch/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 1684, in _finish
    tel.feature.finish = True
  File "/home/mira/anaconda3/envs/gpytorch/lib/pytho

Exception: problem

Error in callback <function _WandbInit._pause_backend at 0x7f91d2f39670> (for post_run_cell):


Exception: The wandb backend process has shutdown

## Skeleton of GP regression model

1. GP Model
2. Likelihood
3. Mean 
4. Kernel
5. Multivate Normal Distirbution

In [7]:
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

In [10]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

model.train()
likelihood.train()

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [ ]:


#wandb.config() #– Saves all your hyperparameters in a config object. 

#wandb.watch()

#wandb.save()